# Synthetic Data Analysis
This notebook is dedicated to exploring and analyzing the synthetic dataset.

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plot
import seaborn as sbn
from pprint import pprint
from typing import Dict, Tuple
from scipy.stats import spearmanr, kendalltau, pearsonr

# Import utility functions
src_path = os.path.abspath(os.path.join('..', 'src'))
if src_path not in sys.path:
    sys.path.append(src_path)

from utils import *
from classes import Plotter

df = pd.read_csv('../data/raw/synthetic/synthetic.csv')

pd.set_option('display.max_columns', None)



In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 34 columns):
 #   Column                        Non-Null Count  Dtype              
---  ------                        --------------  -----              
 0   CVE_ID                        400 non-null    string             
 1   CVE_Published_Date            400 non-null    datetime64[ns, UTC]
 2   Days_To_Publish               400 non-null    Float64            
 3   CVSS_Score                    400 non-null    Float64            
 4   Exploitability_Score          400 non-null    Float64            
 5   Device_Type                   400 non-null    string             
 6   Industry                      400 non-null    string             
 7   Patch_Availability            400 non-null    string             
 8   Impact_Category               400 non-null    string             
 9   MITRE_Tactic                  400 non-null    string             
 10  Relevant_to_ED_SCADA          400 non-

In [66]:
df.head(2)

,CVE_ID,CVE_Published_Date,Days_To_Publish,CVSS_Score,Exploitability_Score,Device_Type,Industry,Patch_Availability,Impact_Category,MITRE_Tactic,Relevant_to_ED_SCADA,Exploit_Code_Date,EPSS_Score_On_Exploit_Date,Operational_Impact_Score,Downtime_Sensitivity,Regulatory_Mandate,Device_Criticality,Time_To_Patch,Patch_Urgency,Patch_Deployment_Success,Exploit_Success_Rate,Known_Exploit_Attempts,Nation_State_Involvement,Threat_Actor,Insider_Access_Required,Supply_Chain_Vulnerability,Remote_Exploitability_SCADA,High_Risk,Low_Risk,Exploited,Risk_Level,Threat Actor Attribution,SecureGrid_Score
0,CVE-2024-4103,2024-11-17,56,8.5,9.0,IED,Power Transmission,No Patch,Data Manipulation,Impact,1,2024-10-25,0.357,7,Low,Yes,3,189,Medium,Yes,0.015,42,No,China,No,No,Medium,1,0,1,1,0,225.01125
1,CVE-2020-5135,2024-10-12,17,7.2,8.5,Smart Meter,Energy Distribution,No Patch,Physical Damage,Data Manipulation,0,2024-10-12,0.394,1,High,Yes,3,151,Low,Yes,0.449,24,No,Iran,No,No,Medium,1,0,1,1,0,45.33675


In [2]:
# Convert types
COL_TYPES = {
    'string': [
        'CVE_ID', 'Device_Type', 'Industry', 'Patch_Availability',
        'Impact_Category', 'MITRE_Tactic', 'Downtime_Sensitivity', 
        'Regulatory_Mandate', 'Patch_Urgency', 'Patch_Deployment_Success',
        'Nation_State_Involvement', 'Threat Actor', 'Insider_Access_Required',
        'Supply_Chain_Vulnerability', 'Remote_Exploitability_SCADA'
    ],
    'date': ['CVE_Published_Date', 'Exploit_Code_Date'],
    'float': [
        'Days_To_Publish', 'CVSS_Score', 'Exploitability_Score',
        'Relevant_to_ED_SCADA', 'EPSS_Score_On_Exploit_Date',
        'Operational_Impact_Score', 'Device_Criticality', 'Time_To_Patch',
        'Exploit_Success_Rate', 'Known_Exploit_Attempts', 'High_Risk',
        'Low_Risk', 'Exploited', 'Risk_Level', 'Threat_Actor_Attribution',
        'SecureGrid_Score'
    ],
}

df = convert_cols(df, COL_TYPES)

# Calculate days to exploit release date
df['Days_To_Exploit_Code_Release'] = (df['Exploit_Code_Date'] - df['CVE_Published_Date']).dt.days
df['Days_To_Exploit_Code_Release'] = df['Days_To_Exploit_Code_Release'].astype('Float64')

CVE_ID converted to string!
Device_Type converted to string!
Industry converted to string!
Patch_Availability converted to string!
Impact_Category converted to string!
MITRE_Tactic converted to string!
Downtime_Sensitivity converted to string!
Regulatory_Mandate converted to string!
Patch_Urgency converted to string!
Patch_Deployment_Success converted to string!
Nation_State_Involvement converted to string!
Insider_Access_Required converted to string!
Supply_Chain_Vulnerability converted to string!
Remote_Exploitability_SCADA converted to string!
CVE_Published_Date converted to datetime!
Exploit_Code_Date converted to datetime!
Days_To_Publish converted to float!
CVSS_Score converted to float!
Exploitability_Score converted to float!
Relevant_to_ED_SCADA converted to float!
EPSS_Score_On_Exploit_Date converted to float!
Operational_Impact_Score converted to float!
Device_Criticality converted to float!
Time_To_Patch converted to float!
Exploit_Success_Rate converted to float!
Known_Exp

In [9]:
# Define dictionary of filtered datasets
filtered_dfs = {
    'full_dataset': df,
    'cvss>=7': df[df['CVSS_Score'] >= 7.0],
    'exploitability>=5': df[df['Exploitability_Score'] >= 5],
    'high_patch_urgency': df[df['Patch_Urgency'] == 'High'],
    'patch_available': df[df['Patch_Availability'] == 'Patch Available'],
    'operation_impact>=7': df[df['Operational_Impact_Score'] >= 7],
    'scada_relevant': df[df['Relevant_to_ED_SCADA'] == 1],
    'days_to_exp_code<0': df[df['Days_To_Exploit_Code_Release'] < 0],
    'days_to_exp_code>=0': df[df['Days_To_Exploit_Code_Release'] >= 0],
    'days_to_exp_code>=0_&_<=90': df[(df['Days_To_Exploit_Code_Release'] >= 0) & (df['Days_To_Exploit_Code_Release'] <= 90)],
    'days_to_exp_code>90': df[df['Days_To_Exploit_Code_Release'] > 90],
    'time_to_patch<0': df[df['Time_To_Patch'] < 0],
    'time_to_patch>=0': df[df['Time_To_Patch'] >= 0],
    'time_to_patch>=0_&_<=90': df[(df['Time_To_Patch'] >= 0) & (df['Time_To_Patch'] <= 90)],
    'time_to_patch>90': df[df['Time_To_Patch'] > 90],
    'high_risk': df[df['High_Risk'] == 1]
}

# Function to check requisite length for correlation test meaningfulness
def correlate(x: pd.Series, y: pd.Series, method: str, precision: int=3):
    df = pd.DataFrame({'x': x, 'y': y})
    if len(df) < 2 or df['x'].nunique() < 2 or df['y'].nunique() < 2:
        return (pd.NA, pd.NA)

    if method == 'pearson':
        coef, p = pearsonr(df['x'], df['y'])
    elif method == 'spearman':
        coef, p = spearmanr(df['x'], df['y'])
    elif method == 'kendall':
        coef, p = kendalltau(df['x'], df['y'])
    else:
        raise ValueError(f'Unknown correlation method: {method.title()}')

    return (round(coef, precision), round(p, precision))

# Function to compute statistics
def compute_stats(df):
    # Columns required specifically for correlations
    corr_cols = [
        'CVSS_Score',
        'Exploitability_Score',
        'EPSS_Score_On_Exploit_Date',
        'Known_Exploit_Attempts',
        'Exploit_Success_Rate',
        'Time_To_Patch',
        'Patch_Urgency',
        'Risk_Level',
        'SecureGrid_Score'
    ]
    # Null-scrubbed dataframe needed for correlation testing
    corr_df = df.dropna(subset=corr_cols).copy()

    # Convert categorical to numerical data for correlation compatibility
    urgency_mapping = {'High': 3, 'Medium': 2, 'Low': 1}
    corr_df['Patch_Urgency'] = corr_df['Patch_Urgency'].map(urgency_mapping)

    return {
        # General facts about the dataset
        'cnt_cve': len(df),
        'cnt_cvss>=7': len(df[df['CVSS_Score'] >= 7]),
        '%_cvss>=7': (len(df[df['CVSS_Score'] >= 7]) / len(df)) * 100 if len(df) > 0 else pd.NA,
        'cnt_exploitability>=5': len(df[df['Exploitability_Score'] >= 5]),
        '%_exploitability>=5': (len(df[df['Exploitability_Score'] >= 5]) / len(df)) * 100 if len(df) > 0 else pd.NA,
        'cnt_high_patch_urgency': len(df[df['Patch_Urgency'] == 'High']),
        '%_high_patch_urgency': (len(df[df['Patch_Urgency'] == 'High']) / len(df)) * 100 if len(df) > 0 else pd.NA,
        'cnt_patch_availability': len(df[df['Patch_Availability'] == 'Patch Available']),
        '%_patch_availability': (len(df[df['Patch_Availability'] == 'Patch Available']) / len(df)) * 100 if len(df) > 0 else pd.NA,
        'cnt_operational_impact>=7': len(df[df['Operational_Impact_Score'] >= 7]),
        '%_operational_impact>=7': (len(df[df['Operational_Impact_Score'] >= 7]) / len(df)) * 100 if len(df) > 0 else pd.NA,

        # Averages
        'avg_days_to_publish': df['Days_To_Publish'].mean(),
        'avg_cvss': df['CVSS_Score'].mean(),
        'avg_exploitability': df['Exploitability_Score'].mean(),
        'avg_patch_availability': df['Patch_Availability'].map({'Patch Available': 1, 'No Patch': 0}).astype('Float64').mean(), # § GOOD
        'avg_scada_relevance': df['Relevant_to_ED_SCADA'].mean(),
        'avg_epss_on_exploit_date': df['EPSS_Score_On_Exploit_Date'].mean(),
        'avg_operational_impact': df['Operational_Impact_Score'].mean(),
        'avg_reg_mandate': df['Regulatory_Mandate'].map({'Yes': 1, 'No': 2}).astype('Float64').mean(), # § GOOD
        'avg_device_criticality': df['Device_Criticality'].mean(),
        'avg_time_to_patch': df['Time_To_Patch'].mean(),
        'avg_patch_success': df['Patch_Deployment_Success'].map({'Yes': 1, 'No': 2}).astype('Float64').mean(), # § GOOD
        'avg_exploit_success': df['Exploit_Success_Rate'].mean(),
        'avg_exploit_attempts': df['Known_Exploit_Attempts'].mean(),
        'avg_state_involvement': df['Nation_State_Involvement'].map({'Yes': 1, 'No': 2}).astype('Float64').mean(), # § GOOD
        'avg_insider_access_req': df['Insider_Access_Required'].map({'Yes': 1, 'No': 2}).astype('Float64').mean(), # § GOOD
        'avg_supply_chain_vuln': df['Supply_Chain_Vulnerability'].map({'Yes': 1, 'No': 2}).astype('Float64').mean(), # § GOOD
        'avg_risk_level': df['Risk_Level'].mean(),
        'avg_threat_actor_attribution': df['Threat Actor Attribution'].mean(),
        'avg_securegrid_score': df['SecureGrid_Score'].mean(),

        # Correlations of CVSS
        'pearson_cvss_vs_exploitability_score': correlate(corr_df['CVSS_Score'], corr_df['Exploitability_Score'], 'pearson'),
        'spearman_cvss_vs_exploitability_score': correlate(corr_df['CVSS_Score'], corr_df['Exploitability_Score'], 'spearman'),
        'kendall_cvss_vs_exploitability_score': correlate(corr_df['CVSS_Score'], corr_df['Exploitability_Score'], 'kendall'),
        'pearson_cvss_vs_exploit_attempts': correlate(corr_df['CVSS_Score'], corr_df['Known_Exploit_Attempts'], 'pearson'),
        'spearman_cvss_vs_exploit_attempts': correlate(corr_df['CVSS_Score'], corr_df['Known_Exploit_Attempts'], 'spearman'),
        'kendall_cvss_vs_exploit_attempts': correlate(corr_df['CVSS_Score'], corr_df['Known_Exploit_Attempts'], 'kendall'),
        'pearson_cvss_vs_exploit_success': correlate(corr_df['CVSS_Score'], corr_df['Exploit_Success_Rate'], 'pearson'),
        'spearman_cvss_vs_exploit_success': correlate(corr_df['CVSS_Score'], corr_df['Exploit_Success_Rate'], 'spearman'),
        'kendall_cvss_vs_exploit_success': correlate(corr_df['CVSS_Score'], corr_df['Exploit_Success_Rate'], 'kendall'),
        'pearson_cvss_vs_time2patch': correlate(corr_df['CVSS_Score'], corr_df['Time_To_Patch'], 'pearson'),
        'spearman_cvss_vs_time2patch': correlate(corr_df['CVSS_Score'], corr_df['Time_To_Patch'], 'spearman'),
        'kendall_cvss_vs_time2patch': correlate(corr_df['CVSS_Score'], corr_df['Time_To_Patch'], 'kendall'),
        'pearson_cvss_vs_risk': correlate(corr_df['CVSS_Score'], corr_df['Risk_Level'], 'pearson'),
        'spearman_cvss_vs_risk': correlate(corr_df['CVSS_Score'], corr_df['Risk_Level'], 'spearman'),
        'kendall_cvss_vs_risk': correlate(corr_df['CVSS_Score'], corr_df['Risk_Level'], 'kendall'),
        'pearson_cvss_vs_epss': correlate(corr_df['CVSS_Score'], corr_df['EPSS_Score_On_Exploit_Date'], 'pearson'),
        'spearman_cvss_vs_epss': correlate(corr_df['CVSS_Score'], corr_df['EPSS_Score_On_Exploit_Date'], 'spearman'),
        'kendall_cvss_vs_epss': correlate(corr_df['CVSS_Score'], corr_df['EPSS_Score_On_Exploit_Date'], 'kendall'),
        'pearson_cvss_vs_patch_urgency': correlate(corr_df['CVSS_Score'], corr_df['Patch_Urgency'], 'pearson'),
        'spearman_cvss_vs_patch_urgency': correlate(corr_df['CVSS_Score'], corr_df['Patch_Urgency'], 'spearman'),
        'kendall_cvss_vs_patch_urgency': correlate(corr_df['CVSS_Score'], corr_df['Patch_Urgency'], 'kendall'),

        # Correlation of CVSS with SecureGrid
        'pearson_cvss_vs_secgrid': correlate(corr_df['CVSS_Score'], corr_df['SecureGrid_Score'], 'pearson'),
        'spearman_cvss_vs_secgrid': correlate(corr_df['CVSS_Score'], corr_df['SecureGrid_Score'], 'spearman'),
        'kendall_cvss_vs_secgrid': correlate(corr_df['CVSS_Score'], corr_df['SecureGrid_Score'], 'kendall'),

        # Correlations of Securegrid
        'pearson_secgrid_vs_exploitability_score': correlate(corr_df['SecureGrid_Score'], corr_df['Exploitability_Score'], 'pearson'),
        'spearman_secgrid_vs_exploitability_score': correlate(corr_df['SecureGrid_Score'], corr_df['Exploitability_Score'], 'spearman'),
        'kendall_secgrid_vs_exploitability_score': correlate(corr_df['SecureGrid_Score'], corr_df['Exploitability_Score'], 'kendall'),
        'pearson_secgrid_vs_exploit_attempts': correlate(corr_df['SecureGrid_Score'], corr_df['Known_Exploit_Attempts'], 'pearson'),
        'spearman_secgrid_vs_exploit_attempts': correlate(corr_df['SecureGrid_Score'], corr_df['Known_Exploit_Attempts'], 'spearman'),
        'kendall_secgrid_vs_exploit_attempts': correlate(corr_df['SecureGrid_Score'], corr_df['Known_Exploit_Attempts'], 'kendall'),
        'pearson_secgrid_vs_exploit_success': correlate(corr_df['SecureGrid_Score'], corr_df['Exploit_Success_Rate'], 'pearson'),
        'spearman_secgrid_vs_exploit_success': correlate(corr_df['SecureGrid_Score'], corr_df['Exploit_Success_Rate'], 'spearman'),
        'kendall_secgrid_vs_exploit_success': correlate(corr_df['SecureGrid_Score'], corr_df['Exploit_Success_Rate'], 'kendall'),
        'pearson_secgrid_vs_time2patch': correlate(corr_df['SecureGrid_Score'], corr_df['Time_To_Patch'], 'pearson'),
        'spearman_secgrid_vs_time2patch': correlate(corr_df['SecureGrid_Score'], corr_df['Time_To_Patch'], 'spearman'),
        'kendall_secgrid_vs_time2patch': correlate(corr_df['SecureGrid_Score'], corr_df['Time_To_Patch'], 'kendall'),
        'pearson_secgrid_vs_risk': correlate(corr_df['SecureGrid_Score'], corr_df['Risk_Level'], 'pearson'),
        'spearman_secgrid_vs_risk': correlate(corr_df['SecureGrid_Score'], corr_df['Risk_Level'], 'spearman'),
        'kendall_secgrid_vs_risk': correlate(corr_df['SecureGrid_Score'], corr_df['Risk_Level'], 'kendall'),
        'pearson_secgrid_vs_epss': correlate(corr_df['SecureGrid_Score'], corr_df['EPSS_Score_On_Exploit_Date'], 'pearson'),
        'spearman_secgrid_vs_epss': correlate(corr_df['SecureGrid_Score'], corr_df['EPSS_Score_On_Exploit_Date'], 'spearman'),
        'kendall_secgrid_vs_epss': correlate(corr_df['SecureGrid_Score'], corr_df['EPSS_Score_On_Exploit_Date'], 'kendall'),
        'pearson_secgrid_vs_patch_urgency': correlate(corr_df['SecureGrid_Score'], corr_df['Patch_Urgency'], 'pearson'),
        'spearman_secgrid_vs_patch_urgency': correlate(corr_df['SecureGrid_Score'], corr_df['Patch_Urgency'], 'spearman'),
        'kendall_secgrid_vs_patch_urgency': correlate(corr_df['SecureGrid_Score'], corr_df['Patch_Urgency'], 'kendall'),
    }

# Loop through each filtered dataframe to calculate stats
stats_list = []
for filter_name, filtered_df in filtered_dfs.items():
    print(f'Computing stats for {filter_name}...')
    stats = compute_stats(filtered_df)
    stats['filter_name'] = filter_name
    stats_list.append(stats)

# Convert stats to DataFrame
summary_df = pd.DataFrame(stats_list)

remainder_cols = [
    col for col in summary_df.columns
    if col != 'filter_name'
]
ordered_cols = ['filter_name'] + remainder_cols
summary_df = summary_df[ordered_cols]

# Output the resulting DataFrame
summary_df.to_csv('../data/raw/synthetic/synthetic_data_stats.csv', index=None)

summary_df.head(5)

Computing stats for full_dataset...
Computing stats for cvss>=7...
Computing stats for exploitability>=5...
Computing stats for high_patch_urgency...
Computing stats for patch_available...
Computing stats for operation_impact>=7...
Computing stats for scada_relevant...
Computing stats for days_to_exp_code<0...
Computing stats for days_to_exp_code>=0...
Computing stats for days_to_exp_code>=0_&_<=90...
Computing stats for days_to_exp_code>90...
Computing stats for time_to_patch<0...
Computing stats for time_to_patch>=0...
Computing stats for time_to_patch>=0_&_<=90...
Computing stats for time_to_patch>90...
Computing stats for high_risk...


,filter_name,cnt_cve,cnt_cvss>=7,%_cvss>=7,cnt_exploitability>=5,%_exploitability>=5,cnt_high_patch_urgency,%_high_patch_urgency,cnt_patch_availability,%_patch_availability,cnt_operational_impact>=7,%_operational_impact>=7,avg_days_to_publish,avg_cvss,avg_exploitability,avg_patch_availability,avg_scada_relevance,avg_epss_on_exploit_date,avg_operational_impact,avg_reg_mandate,avg_device_criticality,avg_time_to_patch,avg_patch_success,avg_exploit_success,avg_exploit_attempts,avg_state_involvement,avg_insider_access_req,avg_supply_chain_vuln,avg_risk_level,avg_threat_actor_attribution,avg_securegrid_score,pearson_cvss_vs_exploitability_score,spearman_cvss_vs_exploitability_score,kendall_cvss_vs_exploitability_score,pearson_cvss_vs_exploit_attempts,spearman_cvss_vs_exploit_attempts,kendall_cvss_vs_exploit_attempts,pearson_cvss_vs_exploit_success,spearman_cvss_vs_exploit_success,kendall_cvss_vs_exploit_success,pearson_cvss_vs_time2patch,spearman_cvss_vs_time2patch,kendall_cvss_vs_time2patch,pearson_cvss_vs_risk,spearman_cvss_vs_risk,kendall_cvss_vs_risk,pearson_cvss_vs_epss,spearman_cvss_vs_epss,kendall_cvss_vs_epss,pearson_cvss_vs_patch_urgency,spearman_cvss_vs_patch_urgency,kendall_cvss_vs_patch_urgency,pearson_cvss_vs_secgrid,spearman_cvss_vs_secgrid,kendall_cvss_vs_secgrid,pearson_secgrid_vs_exploitability_score,spearman_secgrid_vs_exploitability_score,kendall_secgrid_vs_exploitability_score,pearson_secgrid_vs_exploit_attempts,spearman_secgrid_vs_exploit_attempts,kendall_secgrid_vs_exploit_attempts,pearson_secgrid_vs_exploit_success,spearman_secgrid_vs_exploit_success,kendall_secgrid_vs_exploit_success,pearson_secgrid_vs_time2patch,spearman_secgrid_vs_time2patch,kendall_secgrid_vs_time2patch,pearson_secgrid_vs_risk,spearman_secgrid_vs_risk,kendall_secgrid_vs_risk,pearson_secgrid_vs_epss,spearman_secgrid_vs_epss,kendall_secgrid_vs_epss,pearson_secgrid_vs_patch_urgency,spearman_secgrid_vs_patch_urgency,kendall_secgrid_vs_patch_urgency
0,full_dataset,400,282,70.5,343,85.75,112,28.0,240,60.0,155,38.75,91.665,7.63325,6.7655,0.6,0.7075,0.28196,5.75,1.515,3.365,149.24,1.285,0.494672,24.32,1.7225,1.825,1.805,0.705,0.277500,196.633504,"(0.028, 0.649)","(0.048, 0.423)","(0.037, 0.42)","(0.03, 0.626)","(0.013, 0.827)","(0.009, 0.827)","(-0.067, 0.267)","(-0.103, 0.088)","(-0.076, 0.078)","(0.048, 0.424)","(0.057, 0.35)","(0.041, 0.344)","(0.75, 0.0)","(0.798, 0.0)","(0.694, 0.0)","(-0.072, 0.233)","(-0.055, 0.364)","(-0.039, 0.361)","(-0.036, 0.555)","(-0.047, 0.442)","(-0.04, 0.42)","(0.083, 0.171)","(0.065, 0.283)","(0.044, 0.305)","(-0.061, 0.317)","(-0.046, 0.444)","(-0.035, 0.419)","(0.049, 0.42)","(0.06, 0.321)","(0.043, 0.293)","(-0.014, 0.813)","(0.009, 0.882)","(0.015, 0.719)","(0.032, 0.593)","(0.037, 0.537)","(0.028, 0.493)","(0.044, 0.469)","(0.059, 0.331)","(0.048, 0.33)","(0.01, 0.872)","(0.002, 0.973)","(0.002, 0.953)","(-0.005, 0.931)","(-0.005, 0.936)","(-0.004, 0.936)"
1,cvss>=7,282,282,100.0,241,85.460993,79,28.014184,173,61.347518,112,39.716312,93.060284,8.329078,6.819504,0.613475,0.723404,0.272464,5.812057,1.503546,3.365248,149.085106,1.276596,0.485606,24.046099,1.716312,1.815603,1.812057,1.0,0.283688,198.644347,"(-0.043, 0.554)","(-0.038, 0.601)","(-0.031, 0.581)","(0.029, 0.69)","(0.021, 0.77)","(0.016, 0.76)","(-0.028, 0.696)","(-0.032, 0.656)","(-0.026, 0.618)","(0.091, 0.205)","(0.108, 0.133)","(0.08, 0.128)","(<NA>, <NA>)","(<NA>, <NA>)","(<NA>, <NA>)","(0.039, 0.586)","(0.024, 0.736)","(0.017, 0.74)","(-0.109, 0.132)","(-0.11, 0.128)","(-0.094, 0.125)","(0.028, 0.697)","(-0.002, 0.979)","(0.001, 0.983)","(-0.063, 0.382)","(-0.045, 0.532)","(-0.035, 0.499)","(0.061, 0.402)","(0.079, 0.275)","(0.057, 0.246)","(0.004, 0.96)","(0.023, 0.748)","(0.025, 0.599)","(-0.043, 0.551)","(-0.047, 0.511)","(-0.029, 0.556)","(<NA>, <NA>)","(<NA>, <NA>)","(<NA>, <NA>)","(-0.042, 0.565)","(-0.047, 0.515)","(-0.03, 0.528)","(-0.011, 0.883)","(-0.017, 0.815)","(-0.013, 0.822)"
2,exploitability>=5,343,241,70.262391,343,100.0,94,27.